# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [159]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf, isnan, count, when, desc, sort_array, asc, avg, lag, floor
from pyspark.sql.functions import concat, lit, avg, split, isnan, when, count, col, sum, mean, stddev, min, max, round
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as Fsum
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.sql.types import DoubleType
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline 
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import functions as sF

In [4]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Identify Features") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [12]:
# load data
df = spark.read.json('mini_sparkify_event_data.json')

In [18]:
# explore first entry
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [19]:
# determine how many observations are in the dataset
df.count()

286500

In [20]:
# drop NA
df = df.dropna(how = "any", subset = ["userId", "sessionId"])
df = df.filter(df.userId!="")

In [22]:
# temporarily drop duplicates to see unique User IDs
df.select('userId').dropDuplicates().sort('userId').show(5)

+------+
|userId|
+------+
|    10|
|   100|
|100001|
|100002|
|100003|
+------+
only showing top 5 rows



In [23]:
# temporarily drop duplicates to see unique SessionIDs
df.select('sessionId').dropDuplicates().sort('sessionId').show(5)

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
+---------+
only showing top 5 rows



In [24]:
# check how many observations remain in dataset after missing values dropped
df.count()

278154

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [25]:
# examine scheme of data
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [26]:
# check first three entries of data
df.take(3)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [28]:
# check how many unique userID's and sessionID' are in the dataset?
print(df.select("userId").dropDuplicates().count())
print(df.select("sessionId").dropDuplicates().count())

225
2312


In [32]:
# check which actions have been recorded
df.select("page").dropDuplicates().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



In [35]:
# see five userID's that churney
df.filter(df.page=="Cancellation Confirmation").select("userId").dropDuplicates().show(5)

+------+
|userId|
+------+
|   125|
|    51|
|    54|
|100014|
|   101|
+------+
only showing top 5 rows



In [36]:
# include new column in data set indicating whether user churned or not
users_churn = df.filter(df.page=="Cancellation Confirmation").select("userId").dropDuplicates()
churn_list = [(row['userId']) for row in users_churn.collect()]
df_withchurn = df.withColumn("churn", df.userId.isin(churn_list))

In [46]:
# see match between ID and whether user churned or not
df_withchurn.select(['userId', 'churn']).dropDuplicates().show(5)

+------+-----+
|userId|churn|
+------+-----+
|    19|false|
|100005| true|
|200007|false|
|300007| true|
|    50|false|
+------+-----+
only showing top 5 rows



In [49]:
# observe activities of user 100005 
df_withchurn.select(['userId', 'firstName', 'page', 'churn']).where(df.userId == 100005).show(20)

+------+---------+----------+-----+
|userId|firstName|      page|churn|
+------+---------+----------+-----+
|100005|    Cason|      Home| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason| Thumbs Up| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|   Upgrade| true|
|100005|    Cason|Add Friend| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|  NextSong| true|
|100005|    Cason|   Upgrade| true|
|100005|    Cason|      Home| true|
+------+---------+----------+-----+
only showing top 20 rows



In [42]:
# observe how many user in the data set churned or not
df_withchurn.dropDuplicates(['userId']).groupby(['churn']).count().show()

+-----+-----+
|churn|count|
+-----+-----+
| true|   52|
|false|  173|
+-----+-----+



In [39]:
# see whether the gender has an impact on churn likelihood
df_withchurn.dropDuplicates(["userId", "gender"]).groupby(["churn", "gender"]).count().sort("churn").show()

+-----+------+-----+
|churn|gender|count|
+-----+------+-----+
|false|     M|   89|
|false|     F|   84|
| true|     F|   20|
| true|     M|   32|
+-----+------+-----+



In [40]:
# a small difference in churn raters between male and female users
churn_rate_f = 20/(20+84) 
churn_rate_m = 32/(32+89)
print("churn rate of female is ",churn_rate_f)
print("churn rate of male is ",churn_rate_m)

churn rate of female is  0.19230769230769232
churn rate of male is  0.2644628099173554


In [43]:
# see whether the level (free, paid) has an impact on churn likelihood
df_withchurn.dropDuplicates(['userId', 'level']).groupby(['churn', 'level']).count().show()

+-----+-----+-----+
|churn|level|count|
+-----+-----+-----+
|false| free|  149|
|false| paid|  129|
| true| paid|   36|
| true| free|   46|
+-----+-----+-----+



In [58]:
# tend to be just a very small difference between free and paying users
churn_rate_free = 46/(46+149) 
churn_rate_paid = 36/(36+129)
print("churn rate of free is ",churn_rate_free)
print("churn rate of paid is ",churn_rate_paid)

churn rate of free is  0.2358974358974359
churn rate of paid is  0.21818181818181817


In [76]:
# average item in session for all pages and users that churned
df_withchurn.where(df_withchurn.churn == 1).groupby(['page']).mean("itemInSession").sort("page").show(40)

+--------------------+------------------+
|                page|avg(itemInSession)|
+--------------------+------------------+
|               About|116.53571428571429|
|          Add Friend| 96.78616352201257|
|     Add to Playlist|117.18015414258188|
|              Cancel|  87.1923076923077|
|Cancellation Conf...|  88.1923076923077|
|           Downgrade|132.53709198813056|
|               Error|          94.09375|
|                Help|  98.6234309623431|
|                Home| 92.65430622009569|
|              Logout|104.60397830018083|
|            NextSong|111.85882288289278|
|         Roll Advert| 55.62668045501551|
|       Save Settings|  77.3103448275862|
|            Settings| 94.04814814814814|
|    Submit Downgrade|100.33333333333333|
|      Submit Upgrade|           35.6875|
|         Thumbs Down|  99.2399193548387|
|           Thumbs Up|111.39483593329747|
|             Upgrade|            42.625|
+--------------------+------------------+



In [92]:
# average item in session for all pages and users that did not churn
df_withchurn.where(df_withchurn.churn == 0).groupby(['page']).mean("itemInSession").sort("page").show(40)

+----------------+------------------+
|            page|avg(itemInSession)|
+----------------+------------------+
|           About| 129.1617312072893|
|      Add Friend|117.19417742378468|
| Add to Playlist|121.53972303206997|
|       Downgrade|126.84633294528521|
|           Error|111.37272727272727|
|            Help|118.95637860082304|
|            Home| 98.26741973840666|
|          Logout|108.69921436588103|
|        NextSong|117.60440030462043|
|     Roll Advert|55.113283884018884|
|   Save Settings| 96.67460317460318|
|        Settings| 114.0088424437299|
|Submit Downgrade|118.27777777777777|
|  Submit Upgrade| 55.06299212598425|
|     Thumbs Down|112.96243902439025|
|       Thumbs Up|118.03301533857089|
|         Upgrade| 55.96124031007752|
+----------------+------------------+



In [86]:
# count of interactions for all pages and users that did churn
df_withchurn.where(df_withchurn.churn == 1).groupby(['page']).count().sort("page").show(40)

+--------------------+-----+
|                page|count|
+--------------------+-----+
|               About|   56|
|          Add Friend|  636|
|     Add to Playlist| 1038|
|              Cancel|   52|
|Cancellation Conf...|   52|
|           Downgrade|  337|
|               Error|   32|
|                Help|  239|
|                Home| 1672|
|              Logout|  553|
|            NextSong|36394|
|         Roll Advert|  967|
|       Save Settings|   58|
|            Settings|  270|
|    Submit Downgrade|    9|
|      Submit Upgrade|   32|
|         Thumbs Down|  496|
|           Thumbs Up| 1859|
|             Upgrade|  112|
+--------------------+-----+



In [84]:
# count of interactions for all pages and users that did not churn
df_withchurn.where(df_withchurn.churn == 0).groupby(['page']).count().sort("page").show(40)

+----------------+------+
|            page| count|
+----------------+------+
|           About|   439|
|      Add Friend|  3641|
| Add to Playlist|  5488|
|       Downgrade|  1718|
|           Error|   220|
|            Help|  1215|
|            Home|  8410|
|          Logout|  2673|
|        NextSong|191714|
|     Roll Advert|  2966|
|   Save Settings|   252|
|        Settings|  1244|
|Submit Downgrade|    54|
|  Submit Upgrade|   127|
|     Thumbs Down|  2050|
|       Thumbs Up| 10692|
|         Upgrade|   387|
+----------------+------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [136]:
# create feature gender
gender = df_withchurn.dropDuplicates(['userId']).sort('userId').select(['userId','gender'])
gender = gender.replace(['F','M'], ['1', '0'], 'gender')
gender = gender.withColumn('gender', gender.gender.cast("int"))
gender.show(5)

+------+------+
|userId|gender|
+------+------+
|    10|     0|
|   100|     0|
|100001|     1|
|100002|     1|
|100003|     1|
+------+------+
only showing top 5 rows



In [133]:
# create feature level
level = df_withchurn.dropDuplicates(['userId']).sort('userId').select(['userId','level'])
level = level.replace(['paid','free'], ['1', '0'], 'level')
level= level.withColumn('level', level.level.cast("int"))
level.show(5)

+------+-----+
|userId|level|
+------+-----+
|    10|    1|
|   100|    1|
|100001|    0|
|100002|    1|
|100003|    0|
+------+-----+
only showing top 5 rows



In [103]:
# create feature listening duration
length_song = df_withchurn.filter(df_withchurn.page=='NextSong').select('userId', 'sessionId', 'length')
length_song = length_song.withColumn('hours', (length_song.length / 3600))
length_song = length_song.groupBy('userId', 'sessionId').sum('hours')
length_song = length_song.groupBy('userId').agg(sF.avg('sum(hours)').alias('mean_hours'), sF.stddev('sum(hours)').alias('stdev_hours')).na.fill(0)
length_song.show(5)

+------+------------------+------------------+
|userId|        mean_hours|       stdev_hours|
+------+------------------+------------------+
|100010|2.6563848154761907|1.5940891633984364|
|200002|       4.352262775| 3.481087253628621|
|   125|0.5803091944444445|               0.0|
|    51|14.535440077777778|11.600663366034107|
|   124|10.042778698313494| 9.370878101268762|
+------+------------------+------------------+
only showing top 5 rows



In [104]:
# create feature number of songs
number_songs = df_withchurn.filter(df_withchurn.page == 'NextSong').groupBy('userId', 'sessionId').count()
number_songs = number_songs.groupBy('userId').agg(sF.avg('count').alias('mean_songs'), sF.stddev('count').alias('stdev_songs'))
number_songs = number_songs.na.fill(0)
number_songs.show(5)

+------+------------------+------------------+
|userId|        mean_songs|       stdev_songs|
+------+------------------+------------------+
|100010|39.285714285714285| 22.96166577663945|
|200002|              64.5|51.309843110264914|
|   125|               8.0|               0.0|
|    51|             211.1| 168.1483802413148|
|   124|145.67857142857142|137.40587446912284|
+------+------------------+------------------+
only showing top 5 rows



In [112]:
# create feature thumbsup
thumbsup = df_withchurn.where(df_withchurn.page=='Thumbs Up').groupby(['userId']).agg(count(col('page')).alias('ThumbsUps')).orderBy('userId').select(['userId','ThumbsUps'])
thumbsup.show(5)

+------+---------+
|userId|ThumbsUps|
+------+---------+
|    10|       37|
|   100|      148|
|100001|        8|
|100002|        5|
|100003|        3|
+------+---------+
only showing top 5 rows



In [119]:
# create feature thumbsdown
thumbsdown = df_withchurn.where(df_withchurn.page=='Thumbs Down').groupby(['userId']).agg(count(col('page')).alias('ThumbsDowns')).orderBy('userId').select(['userId','ThumbsDowns'])
thumbsdown.show(5)

+------+-----------+
|userId|ThumbsDowns|
+------+-----------+
|    10|          4|
|   100|         27|
|100001|          2|
|100004|         11|
|100005|          3|
+------+-----------+
only showing top 5 rows



In [108]:
# create feature songs added to playlist
playlist = df_withchurn.select('userID','page').where(df_withchurn.page == 'Add to Playlist').groupBy('userID').count().withColumnRenamed('count', 'add_to_playlist')
playlist.show(5)

+------+---------------+
|userID|add_to_playlist|
+------+---------------+
|100010|              7|
|200002|              8|
|    51|             52|
|   124|            118|
|     7|              5|
+------+---------------+
only showing top 5 rows



In [109]:
# create feature friends added
friends = df_withchurn.select('userID','page').where(df_withchurn.page == 'Add Friend').groupBy('userID').count().withColumnRenamed('count', 'add_friend')
friends.show(5)

+------+----------+
|userID|add_friend|
+------+----------+
|100010|         4|
|200002|         4|
|    51|        28|
|   124|        74|
|     7|         1|
+------+----------+
only showing top 5 rows



In [124]:
# create binary label churn 
user_churn = df_withchurn.select("userId", "churn").dropDuplicates()
user_churn = user_churn.select("userId", user_churn.churn.cast("int"))

In [137]:
# join all created features to one dataset
df_modeling = df_withchurn.dropDuplicates(['userId']).sort('userId').select(['userId'])
for feature in [gender, level,length_song,number_songs,thumbsup,thumbsdown,playlist,friends,user_churn]:
    df_modeling = df_modeling.join(feature,'userId')

In [138]:
# check one observation
df_modeling.head()

Row(userId='100010', gender=1, level=0, mean_hours=2.6563848154761907, stdev_hours=1.5940891633984364, mean_songs=39.285714285714285, stdev_songs=22.96166577663945, ThumbsUps=17, ThumbsDowns=5, add_to_playlist=7, add_friend=4, churn=0)

In [ ]:
# data ready for modeling
save_path = "modeling_data.csv"
df_modeling.write.save(save_path, format="csv", header=True)

In [139]:
# check column types
df_modeling.dtypes

[('userId', 'string'),
 ('gender', 'int'),
 ('level', 'int'),
 ('mean_hours', 'double'),
 ('stdev_hours', 'double'),
 ('mean_songs', 'double'),
 ('stdev_songs', 'double'),
 ('ThumbsUps', 'bigint'),
 ('ThumbsDowns', 'bigint'),
 ('add_to_playlist', 'bigint'),
 ('add_friend', 'bigint'),
 ('churn', 'int')]

In [151]:
# use assembler for scaling preparation
assembler = VectorAssembler(inputCols=df_modeling.columns[1:-1], outputCol="NumFeatures")
features = assembler.transform(df_modeling)

In [152]:
# scale dater using StandardScaler
scaler = StandardScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures", withStd=True)
scalerModel = scaler.fit(features)
features = scalerModel.transform(features)
features = features.select(data.churn.alias("label"), features.ScaledNumFeatures.alias("features"))

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [199]:
train, validation = features.randomSplit([0.6, 0.4], seed=1)
#validation, test = val_test.randomSplit([0.5, 0.5], seed=1)

In [200]:
# initialize log regression as classifier
lr = LogisticRegression(maxIter=10)

# set evaluator
f1_evaluator = MulticlassClassificationEvaluator(metricName='f1')

# build paramGrid
paramGrid = ParamGridBuilder().build()

# set up cross validation logistic regression
crossval_lr = CrossValidator(estimator=lr,evaluator=f1_evaluator, estimatorParamMaps=paramGrid,numFolds=3)

In [201]:
# train lr model
cvModel_lr = crossval_lr.fit(train)

In [202]:
# predict with lr model
results_lr = cvModel_lr.transform(validation)

In [203]:
# ceck results_lr data
results_lr

DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [204]:
# print evaluation for lr performance
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
print('Logistic Regression Metrics:')
print('Accuracy: {}'.format(evaluator.evaluate(results_lr, {evaluator.metricName: "accuracy"})))
print('F-1 Score:{}'.format(evaluator.evaluate(results_lr, {evaluator.metricName: "f1"})))

Logistic Regression Metrics:
Accuracy: 0.7857142857142857
F-1 Score:0.6914285714285714


In [205]:
# initialize random forest as classifier
rf = RandomForestClassifier()

# set evaluator
f1_evaluator = MulticlassClassificationEvaluator(metricName='f1')

# build paramGrid
paramGrid = ParamGridBuilder() \
    .build()

# set up cross validation random forest
crossval_rf = CrossValidator(estimator=rf,estimatorParamMaps=paramGrid,evaluator=f1_evaluator,numFolds=3)

In [206]:
# train rf model
cvModel_rf = crossval_rf.fit(train)

In [207]:
# predict using rf model
results_rf = cvModel_rf.transform(validation)

In [208]:
# print evaluation for rf performance
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
print('Random Forest Metrics:')
print('Accuracy: {}'.format(evaluator.evaluate(results_rf, {evaluator.metricName: "accuracy"})))
print('F-1 Score:{}'.format(evaluator.evaluate(results_rf, {evaluator.metricName: "f1"})))

Random Forest Metrics:
Accuracy: 0.7428571428571429
F-1 Score:0.6892857142857144


In [209]:
# initialize support vector machine as classifier
svm = LinearSVC(maxIter=10)

# set evaluator
f1_evaluator = MulticlassClassificationEvaluator(metricName='f1')

# build paramGrid
paramGrid = ParamGridBuilder() \
    .build()

# set up cross validation for SVM
crossval_svm = CrossValidator(estimator=svm,estimatorParamMaps=paramGrid,evaluator=f1_evaluator,numFolds=3)

In [211]:
# train svm model
cvModel_svm = crossval_svm.fit(train)

In [212]:
# predict using svm model
results_svm = cvModel_svm.transform(validation)

In [213]:
# print evaluation for svm performance
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
print('SVM Metrics:')
print('Accuracy: {}'.format(evaluator.evaluate(results_svm, {evaluator.metricName: "accuracy"})))
print('F-1 Score:{}'.format(evaluator.evaluate(results_svm, {evaluator.metricName: "f1"})))

SVM Metrics:
Accuracy: 0.7857142857142857
F-1 Score:0.6914285714285714


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.